In [1]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
import numpy as np

data_path = "G:\ABCD\script/trail/trail_tsne_RF"
# load data and drop the first column and the subject id
data = pd.read_csv(data_path + "/merged.csv").drop(columns=["Unnamed: 0", "src_subject_id"])
label_columns = data.columns[data.columns.str.startswith("cbcl")].tolist()

data = data[label_columns]

# EFA from previous research

In [44]:
# delete columns with low frequency (more than 99.5% of the values are 0)
low_frequency_columns = data.columns[data.apply(lambda col: (col == 0).mean() > 0.995)]
data_cleaned = data.drop(columns=low_frequency_columns)
print(f"Removed columns with low frequency: {low_frequency_columns.tolist()}")

#load corerlation matrix from polychoric correlation matrix

correlation_matrix = pd.read_csv(data_path + "/factor analysis/polychoric_correlation_matrix.csv", index_col=0)

# mark highly correlated pairs (r > 0.75)
high_corr_pairs = (correlation_matrix.abs() > 0.75).where(lambda x: np.triu(x, 1)).stack().index.tolist()
print(f"Highly correlated pairs (r > 0.75): {high_corr_pairs}")

Removed columns with low frequency: ['cbcl_q02_p', 'cbcl_q73_p', 'cbcl_q99_p', 'cbcl_q101_p', 'cbcl_q105_p']
Highly correlated pairs (r > 0.75): [('cbcl_q08_p', 'cbcl_q10_p'), ('cbcl_q08_p', 'cbcl_q78_p'), ('cbcl_q20_p', 'cbcl_q21_p'), ('cbcl_q21_p', 'cbcl_q106_p'), ('cbcl_q22_p', 'cbcl_q28_p'), ('cbcl_q23_p', 'cbcl_q28_p'), ('cbcl_q25_p', 'cbcl_q48_p'), ('cbcl_q53_p', 'cbcl_q55_p'), ('cbcl_q56c_p', 'cbcl_q56f_p'), ('cbcl_q57_p', 'cbcl_q97_p'), ('cbcl_q81_p', 'cbcl_q82_p')]


In [45]:
from collections import defaultdict

def find_connected_groups(pairs):
    # 建立图结构
    graph = defaultdict(set)
    for col1, col2 in pairs:
        graph[col1].add(col2)
        graph[col2].add(col1)
    
    # 深度优先搜索（DFS）找到所有连通分量
    visited = set()
    connected_groups = []

    def dfs(node, group):
        visited.add(node)
        group.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor, group)

    # 遍历所有节点，找到每个连通分量
    for node in graph:
        if node not in visited:
            group = set()
            dfs(node, group)
            connected_groups.append(tuple(sorted(group)))

    return connected_groups

# 使用函数
result = find_connected_groups(high_corr_pairs)
print("number of connected groups:", len(result))
print("columns for each connected group:", result)

number of connected groups: 8
columns for each connected group: [('cbcl_q08_p', 'cbcl_q10_p', 'cbcl_q78_p'), ('cbcl_q106_p', 'cbcl_q20_p', 'cbcl_q21_p'), ('cbcl_q22_p', 'cbcl_q23_p', 'cbcl_q28_p'), ('cbcl_q25_p', 'cbcl_q48_p'), ('cbcl_q53_p', 'cbcl_q55_p'), ('cbcl_q56c_p', 'cbcl_q56f_p'), ('cbcl_q57_p', 'cbcl_q97_p'), ('cbcl_q81_p', 'cbcl_q82_p')]


In [46]:
#create dataframe to store the final data
data_final = data_cleaned.copy()
for group in result:
    # calculate the average of the columns in the group
    data_final[f"avg_{'_'.join(group)}"] = data_cleaned[list(group)].mean(axis=1).round().astype(int)
    # delete the original columns
    data_final.drop(columns=list(group))

In [65]:
# #scale data_final
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(data_final)

# #make data_scaled to dataframe
# data_scaled = pd.DataFrame(data_scaled, columns=data_final.columns)
data_scaled = data_final

In [89]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
import matplotlib.pyplot as plt
import seaborn as sns


# Bartlett 和 KMO 测试
chi_square_value, p_value = calculate_bartlett_sphericity(data_scaled)
print(f"Bartlett's Test Chi-square: {chi_square_value}, p-value: {p_value}")
kmo_all, kmo_model = calculate_kmo(data_scaled)
print(f"KMO Test Score: {kmo_model}")

# factor analysis
fa = FactorAnalyzer(n_factors=16,  rotation="promax", method = 'principal')
# fa = FactorAnalyzer(n_factors=5, rotation="varimax")
# fa.fit(data_cleaned)
fa.fit(data_scaled)

# factor loadings
factor_loadings = fa.loadings_
# factor_loadings_df = pd.DataFrame(factor_loadings, columns=["Factor 1", "Factor 2", "Factor 3", "Factor 4", "Factor 5", "Factor 6"])
factor_loadings_df = pd.DataFrame(factor_loadings, columns=[f"Factor {i}" for i in range(1, 17)])
# factor_loadings_df = pd.DataFrame(factor_loadings, columns=["Factor 1", "Factor 2", "Factor 3", "Factor 4", "Factor 5"])
# factor_loadings_df.index = data_cleaned.columns
factor_loadings_df.index = data_final.columns
print("Factor Loadings DataFrame:\n", factor_loadings_df)

# variance explained
variance_explained = fa.get_factor_variance()
print("Variance Explained:\n", variance_explained)



Bartlett's Test Chi-square: 515315.3694519205, p-value: 0.0
KMO Test Score: 0.94879737086253


c:\Anaconda\envs\PhD\Lib\site-packages\factor_analyzer\utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(


Factor Loadings DataFrame:
                              Factor 1  Factor 2  Factor 3  Factor 4  Factor 5  \
cbcl_q01_p                   0.211555 -0.012814  0.391568 -0.036343  0.016208   
cbcl_q03_p                   0.761260 -0.010440  0.040119 -0.004430  0.000960   
cbcl_q04_p                   0.138168 -0.084475  0.713752  0.025443  0.033466   
cbcl_q05_p                   0.301061 -0.144140  0.099289 -0.026919  0.004506   
cbcl_q06_p                  -0.024992 -0.047099  0.010108 -0.055517  0.012824   
...                               ...       ...       ...       ...       ...   
avg_cbcl_q25_p_cbcl_q48_p   -0.090328 -0.020791 -0.179116  0.013832 -0.013794   
avg_cbcl_q53_p_cbcl_q55_p   -0.008985  0.028284 -0.027917 -0.062178  0.978707   
avg_cbcl_q56c_p_cbcl_q56f_p  0.005110  0.013183 -0.037452  0.974309 -0.025605   
avg_cbcl_q57_p_cbcl_q97_p   -0.069001  0.043082  0.037655  0.056761  0.013820   
avg_cbcl_q81_p_cbcl_q82_p   -0.115884  0.028409  0.005127  0.000674  0.017981   


In [88]:
variance_explained[1]

array([0.05881895, 0.02493378, 0.0632039 , 0.03119876, 0.02296772,
       0.02669731, 0.0360115 , 0.02971527, 0.0287665 , 0.02221154,
       0.0335414 , 0.02619438, 0.0242855 , 0.02263868, 0.01949415,
       0.01297078])